In [1]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier

In [2]:
df=pd.read_csv('Path sample gen.xlsx - Vehicle Routes.csv')

In [3]:
df

,Index,Route ID,Target Time,Time to Next Stop,Stop,Postcode
0,1,RTE01,17:15,0:15,SGH Pathology Lab,SO16 6YD
1,2,RTE01,17:30,0:20,Surgery 61,SO40 3JD
2,3,RTE01,17:50,0:10,Surgery 1,SO51 8XA
3,4,RTE01,18:00,0:10,Surgery 5,SO51 8JH
4,5,RTE01,18:10,0:15,Surgery 63,SO51 7LB
...,...,...,...,...,...,...
179,180,RTE10,12:40,0:20,Surgery 36,SO43 7FJ
180,181,RTE10,13:00,0:05,Surgery 27,SO45 4JG
181,182,RTE10,13:05,0:05,Surgery 72,SO45 5GQ
182,183,RTE10,13:10,0:30,Surgery 71,SO45 6AG


In [18]:
df = df.fillna('')

In [19]:
df['route_id'] = df['Route ID'].factorize()[0]  
df['stop_id'] = df['Stop'].factorize()[0]

In [20]:
df['Target Time'] = df['Target Time'].str.replace(':','')

In [21]:
df['hours'] = df['Target Time'].str[:2].astype(int) 

In [22]:
X = df[['stop_id','hours']]   
y = df['route_id']

In [23]:
knn = KNeighborsClassifier()
knn.fit(X, y)

KNeighborsClassifier()

In [24]:
encoder = {df['Stop'][i]:i for i in range(len(df['Stop']))}

In [25]:
input_list = line.split()
code = input_list[0]  
time = input_list[1]

In [27]:
print(f"Received input: {line}")

Received input: SO16 6YD 08:30


In [35]:
print("Enter pickup postcode and time:")
pickups = []
while True:
    line = input()
    if not line:
        break
    
    # Extract postcode and time from input        
    m = re.match(r"([A-Z]+\d+\s[A-Z]+) (\d\d:\d\d)", line)
    
    # Check if the match is successful
    if m:
        code = m.group(1)
        hours = int(m.group(2)[:2])
        
        # Append pickup  
        pickups.append([encoder[code], hours])
    else:
        print("Invalid input. Please enter the pickup postcode and time in the correct format.")

# Make prediction
if pickups:
    prediction = knn.predict([x[1] for x in pickups])[0]  
    print("Predicted route:", prediction)
else:
    print("No valid input provided.")

Enter pickup postcode and time:
SO16 6YD 08:30
Invalid input. Please enter the pickup postcode and time in the correct format.

No valid input provided.


In [38]:
pip install networkx

Note: you may need to restart the kernel to use updated packages.


In [39]:
import pandas as pd
import networkx as nx

In [40]:
G = nx.Graph()

In [46]:
for route_id, route_data in df.groupby('Route ID'):
    stops = route_data['Stop'].tolist()
    for i in range(len(stops) - 1):
        stop1, stop2 = stops[i], stops[i+1]
        time_to_next_stop = route_data.iloc[i]['Time to Next Stop']
        
        # Convert time format '0:15' to total minutes
        hours, minutes = map(int, time_to_next_stop.split(':'))
        total_minutes = hours * 60 + minutes

        G.add_edge(stop1, stop2, weight=total_minutes)

# Get user input
print("Enter starting stop and ending stop:")
start_stop = input("Starting stop: ")
end_stop = input("Ending stop: ")

# Check if the stops are in the graph
if start_stop not in G.nodes or end_stop not in G.nodes:
    print("Invalid stops. Please enter valid stops.")
else:
    # Find the shortest path
    shortest_path = nx.shortest_path(G, source=start_stop, target=end_stop, weight='weight')
    shortest_distance = nx.shortest_path_length(G, source=start_stop, target=end_stop, weight='weight')

    print("Shortest path:", shortest_path)
    print("Shortest distance:", shortest_distance)

Enter starting stop and ending stop:
Starting stop: SGH Pathology Lab
Ending stop: Surgery 63
Shortest path: ['SGH Pathology Lab', 'Surgery 63']
Shortest distance: 20
